##View of the most important columns FROM the central table, tictac_genes_diseases_associations
##Query by most important columns in the table tictac_genes_diseases_associations

#First, you may want to understand the data in the database by probing values in specific columns for diseases, drugs, and targets

#Explanation:
#Function fetch_unique_column_values:

#Takes the connection, cursor, and column name as arguments.
#Executes a query to fetch unique values from the specified column.
#Converts the results to a data frame.
#Optionally saves the DataFrame to a TSV file.
#Function prompt_for_next_column:

#Prompts the user to enter a column name.
 #   If a valid column name is entered, it fetches the unique values for that column.
  #  If no input is provided, it starts a 20-second timer before closing the connection.
   # If the timer expires, the connection is closed.
    #Function close_connection:

#Closes the MySQL connection and cursor.
#Main Block:
#
 #   Establishes a connection to the MySQL database.
  #  Verifies the connection and prints server information.
   # Creates a cursor object.
    #Defines a list of valid column names.
    #It prompts the user to enter a column name and has a timeout mechanism to close the connection after 20 seconds of inactivity.

#Optionally, it saves the DataFrame to a TSV file named "{column_name}.tsv." The script handles potential connection errors and ensures the database connection is properly closed after the operation.
#Here is a list of columns you may find interesting to probe
   # disease_name
    #tcrdTargetName
    #uniprot
    #CID
    #nct_id
    #doid
    #drug_name
    #target_chembl_id
    #molecule_chembl_id
    #gene_symbol
    #idgTDL
    #disease_target


In [32]:
import mysql.connector as connector
import pandas as pd
import threading

def fetch_unique_column_values(connection, cursor, column_name):
    try:
        # Define the query
        query = f"SELECT DISTINCT {column_name} FROM tictac_genes_diseases_associations;"
        
        # Execute the query
        cursor.execute(query)
        
        # Fetch all unique values for the specified column
        results = cursor.fetchall()
        
        # Convert to DataFrame
        df = pd.DataFrame(results, columns=[column_name])
        print(df)
        
        # Optional: Save to TSV file
        df.to_csv(f"unique_{column_name}.tsv", sep='\t', index=False)
        
    except connector.Error as e:
        print("Error while executing query", e)

def prompt_for_next_column(connection, cursor, valid_columns):
    while True:
        column_name = input("Enter the column name (or press Enter to exit): ")
        
        if column_name:
            if column_name in valid_columns:
                fetch_unique_column_values(connection, cursor, column_name)
            else:
                print("Invalid column name. Please choose from the following list:")
                for col in valid_columns:
                    print(f"- {col}")
        else:
            print("No input provided. Closing the connection in 20 seconds...")
            timer = threading.Timer(20, close_connection, args=[connection, cursor])
            timer.start()
            timer.join()
            break

def close_connection(connection, cursor):
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed due to inactivity.")

if __name__ == "__main__":
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )

        if connection.is_connected():
            # Fetch and print some server information
            db_info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_info)

            # Create a cursor object
            cursor = connection.cursor()

            # Execute a simple query to verify the connection
            cursor.execute("SELECT DATABASE();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)

            # List of valid column names
            valid_columns = [
                "disease_term",
                "disease_name",
                "tcrdTargetName",
                "uniprot",
                "CID",
                "nct_id",
                "doid",
                "drug_name",
                "target_chembl_id",
                "molecule_chembl_id",
                "gene_symbol",
                "idgTDL",
                "disease_target"
            ]

            # Prompt for the next column name
            prompt_for_next_column(connection, cursor, valid_columns)

    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")


Connected to MySQL Server version  8.0.37-cluster
You're connected to database:  ('tictac_db',)


Enter the column name (or press Enter to exit):  


No input provided. Closing the connection in 20 seconds...
MySQL connection is closed due to inactivity.


##Query by Disease
#Here's the Python script named disease_query.py, designed for users to interactively select and retrieve data from a MySQL database focusing on columns related to disease information: disease_term, disease_name, and doid.

#Key Features of the Script
    #Dynamic Column and Term Selection: Users can select from three specific columns (disease_term, disease_name, doid) and then choose a specific term within that column for detailed querying.
    #Data Fetch and Display: The script fetches data matching the selected term and displays it in a console. It also saves the results to a TSV file for offline analysis.
    #User Interface: A graphical interface using Tkinter simplifies the selection process, making the script accessible to users with little to no programming experience.

In [33]:

import mysql.connector as connector
import pandas as pd
import tkinter as tk
from tkinter import ttk

def fetch_unique_terms(column_name):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="Je@#1978",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"SELECT DISTINCT {column_name} FROM tictac_genes_diseases_associations ORDER BY {column_name};"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch all unique terms
            results = cursor.fetchall()
            
            # Convert to list
            unique_terms = [row[0] for row in results]
            return unique_terms
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)
        return []
    
    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            
def fetch_data_for_term(column_name, search_term):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"""
            SELECT * FROM aact_studies WHERE nct_id IN
            (SELECT DISTINCT t.nct_id
                FROM tictac_genes_diseases_associations AS t
                JOIN aact_studies AS s ON t.nct_id = s.nct_id
                WHERE t.{column_name} LIKE '%{search_term}%');
            """
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results
            results = cursor.fetchall()
            
            # Get column names
            column_names = [i[0] for i in cursor.description]
            
            # Convert to DataFrame
            df = pd.DataFrame(results, columns=column_names)
            
            # Display DataFrame
            print(df)
            
            # Save to TSV file
            df.to_csv(f"{search_term}_data.tsv", sep='\t', index=False)
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

def on_select_column(event=None):
    selected_column = column_combobox.get()
    if selected_column:
        unique_terms = fetch_unique_terms(selected_column)
        term_combobox['values'] = unique_terms
        term_combobox.set('')
    else:
        print("No column selected.")

def on_select_term(event=None):
    selected_column = column_combobox.get()
    selected_term = term_combobox.get()
    if selected_column and selected_term:
        fetch_data_for_term(selected_column, selected_term)
    else:
        print("No term selected.")

# Create the main window
root = tk.Tk()
root.title("Disease Query Selector")

# Column options
columns = ['disease_term', 'disease_name', 'doid']

# Create a style for the combobox and button
style = ttk.Style()
style.configure("TCombobox", font=("Helvetica", 12))
style.configure("TButton", font=("Helvetica", 12))

# Create a Label for column selection
column_label = ttk.Label(root, text="Select a Column:", font=("Helvetica", 14))
column_label.pack(pady=10)

# Create a Combobox for column selection
column_combobox = ttk.Combobox(root, values=columns, style="TCombobox")
column_combobox.pack(pady=10, ipadx=20, ipady=10)
column_combobox.bind("<<ComboboxSelected>>", on_select_column)

# Create a Label for term selection
term_label = ttk.Label(root, text="Select a Term:", font=("Helvetica", 14))
term_label.pack(pady=10)

# Create a Combobox for term selection
term_combobox = ttk.Combobox(root, style="TCombobox")
term_combobox.pack(pady=10, ipadx=20, ipady=10)
term_combobox.bind("<<ComboboxSelected>>", on_select_term)

# Create a Button to manually trigger search
search_button = tk.Button(root, text="Search", command=on_select_term, font=("Helvetica", 12))
search_button.pack(pady=10, ipadx=20, ipady=10)

# Run the main loop
root.mainloop()


MySQL connection is closed
        nct_id      study_type  \
0  NCT00023530  Interventional   

                                              source           phase  \
0  National Heart, Lung, and Blood Institute (NHLBI)  Not Applicable   

  overall_status start_date completion_date enrollment  \
0      Completed       None      2001-09-30              

                                      official_title  
0  Blood and Marrow Transplant Clinical Research ...  
MySQL connection is closed
        nct_id      study_type  \
0  NCT00023530  Interventional   

                                              source           phase  \
0  National Heart, Lung, and Blood Institute (NHLBI)  Not Applicable   

  overall_status start_date completion_date enrollment  \
0      Completed       None      2001-09-30              

                                      official_title  
0  Blood and Marrow Transplant Clinical Research ...  
MySQL connection is closed


In [27]:
import mysql.connector as connector

def fetch_targets_for_disease():
    # Prompt the user to enter a disease name
    disease_name = input("Please enter the disease name: ")

    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="Je@#1978",  # Modify as per your MySQL setup
            database="tictac_db"  # Your specific database name
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query to fetch targets related to the entered disease name
            query = """
            SELECT disease_name, tcrdTargetName, uniprot, gene_symbol, target_chembl_id
            FROM tictac_genes_diseases_associations
            WHERE disease_name = %s;
            """
            
            # Execute the query with user input
            cursor.execute(query, (disease_name,))
            
            # Fetch the results
            results = cursor.fetchall()

            if results:
                print(f"Targets associated with {disease_name}:")
                for row in results:
                    print(f"Disease: {row[0]}, Target Name: {row[1]}, UniProt: {row[2]}, Gene Symbol: {row[3]}, ChEMBL ID: {row[4]}")
            else:
                print(f"No targets found associated with the disease: {disease_name}")
    
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    fetch_targets_for_disease()


Please enter the disease name:  schizophrenia


Targets associated with schizophrenia:
Disease: schizophrenia, Target Name: Lysine-specific demethylase 4E, UniProt: B2RXH2, Gene Symbol: KDM4E, ChEMBL ID: CHEMBL1293226
Disease: schizophrenia, Target Name: Lysine-specific demethylase 4E, UniProt: B2RXH2, Gene Symbol: KDM4E, ChEMBL ID: CHEMBL1293226
Disease: schizophrenia, Target Name: Lysine-specific demethylase 4E, UniProt: B2RXH2, Gene Symbol: KDM4E, ChEMBL ID: CHEMBL1293226
Disease: schizophrenia, Target Name: Solute carrier family 22 member 1, UniProt: O15245, Gene Symbol: SLC22A1, ChEMBL ID: CHEMBL5685
Disease: schizophrenia, Target Name: Solute carrier family 22 member 1, UniProt: O15245, Gene Symbol: SLC22A1, ChEMBL ID: CHEMBL5685
Disease: schizophrenia, Target Name: Solute carrier family 22 member 1, UniProt: O15245, Gene Symbol: SLC22A1, ChEMBL ID: CHEMBL5685
Disease: schizophrenia, Target Name: Solute carrier family 22 member 1, UniProt: O15245, Gene Symbol: SLC22A1, ChEMBL ID: CHEMBL5685
Disease: schizophrenia, Target Name:

In [28]:
import mysql.connector as connector
import pandas as pd

def find_disease_with_most_targets():
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            
            # Query to find the disease with the most unique targets
            max_targets_query = """
            SELECT disease_name, COUNT(DISTINCT tcrdTargetName) AS target_count
            FROM tictac_genes_diseases_associations
            GROUP BY disease_name
            ORDER BY target_count DESC
            LIMIT 1;
            """
            
            cursor.execute(max_targets_query)
            max_disease_result = cursor.fetchone()
            
            if max_disease_result:
                disease_name, max_count = max_disease_result
                print(f"Disease with the most targets: {disease_name} with {max_count} unique targets.")
                
                # Query to find the distribution of targets for this disease
                target_distribution_query = f"""
                SELECT tcrdTargetName, COUNT(*) AS count
                FROM tictac_genes_diseases_associations
                WHERE disease_name = %s
                GROUP BY tcrdTargetName
                ORDER BY count DESC;
                """
                
                cursor.execute(target_distribution_query, (disease_name,))
                distribution_results = cursor.fetchall()
                
                # Convert to DataFrame for better visualization
                df = pd.DataFrame(distribution_results, columns=['Target Name', 'Count'])
                print("\nDistribution of targets for", disease_name)
                print(df)
            else:
                print("No data found.")

    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    find_disease_with_most_targets()


Disease with the most targets: anemia with 831 unique targets.

Distribution of targets for anemia
                                          Target Name  Count
0            Histone-lysine N-methyltransferase EHMT2    432
1                                 Cytochrome P450 2D6    260
2                                 Neutrophil elastase    156
3        Receptor-type tyrosine-protein phosphatase C    156
4               Sodium-dependent dopamine transporter    156
..                                                ...    ...
826         Serine/threonine-protein kinase MRCK beta      1
827        Interleukin-1 receptor-associated kinase 3      1
828                Serine/threonine-protein kinase 24      1
829                   Casein kinase I isoform gamma-3      1
830  Mitogen-activated protein kinase kinase kinase 4      1

[831 rows x 2 columns]
MySQL connection is closed


#Query by biological target
#Provenance by Biological Targets
#You may want to find provenance based on biological targets only. Four options are provided for this: tcrdTargetName, uniprot, gene_symbol, and target_chembl_id.
#This script is designed to help users fetch and analyze data from a MySQL database. 
#It starts by connecting to the database to retrieve unique biological targets, which are then displayed in a user-friendly graphical interface using Tkinter. 
#Users can select a biological target from a dropdown menu. Upon selection, the script queries the database for detailed study information related to the chosen target and displays the results in a structured format. 
#Additionally, the results are saved to a TSV file for further analysis. The interface includes a search button to manually trigger the search process, ensuring ease of use and interaction. 
#This script is particularly useful for researchers and analysts working with biological and clinical study data.

#Key Features of the Script
    #Dynamic Column and Term Selection: Users can choose from four specific columns (tcrdTargetName, uniprot, gene_symbol, target_chembl_id), each pertinent to target identification and drug discovery.
    #Data Fetch and Display: Upon term selection, the script retrieves and displays the corresponding data in a console and saves the results to a TSV file for further use.
    #User Interface: A graphical interface using Tkinter simplifies user interaction, making database queries accessible to non-technical users and enhancing usability for biological and pharmaceutical research.


In [31]:
import mysql.connector as connector
import pandas as pd
import tkinter as tk
from tkinter import ttk

def fetch_unique_terms(column_name):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"SELECT DISTINCT {column_name} FROM tictac_genes_diseases_associations ORDER BY {column_name};"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch all unique terms
            results = cursor.fetchall()
            
            # Convert to list
            unique_terms = [row[0] for row in results]
            return unique_terms
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)
        return []
    
    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            
def fetch_data_for_term(column_name, search_term):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"""
            SELECT * FROM aact_studies WHERE nct_id IN
            (SELECT DISTINCT t.nct_id
                FROM tictac_genes_diseases_associations AS t
                JOIN aact_studies AS s ON t.nct_id = s.nct_id
                WHERE t.{column_name} LIKE '%{search_term}%');
            """
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results
            results = cursor.fetchall()
            
            # Get column names
            column_names = [i[0] for i in cursor.description]
            
            # Convert to DataFrame
            df = pd.DataFrame(results, columns=column_names)
            
            # Display DataFrame
            print(df)
            
            # Save to TSV file
            df.to_csv(f"{search_term}_data.tsv", sep='\t', index=False)
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

def on_select_column(event=None):
    selected_column = column_combobox.get()
    if selected_column:
        unique_terms = fetch_unique_terms(selected_column)
        term_combobox['values'] = unique_terms
        term_combobox.set('')
    else:
        print("No column selected.")

def on_select_term(event=None):
    selected_column = column_combobox.get()
    selected_term = term_combobox.get()
    if selected_column and selected_term:
        fetch_data_for_term(selected_column, selected_term)
    else:
        print("No term selected.")

# Create the main window
root = tk.Tk()
root.title("Target Query Selector")

# Column options
columns = ['tcrdTargetName', 'uniprot', 'gene_symbol', 'target_chembl_id']

# Create a style for the combobox and button
style = ttk.Style()
style.configure("TCombobox", font=("Helvetica", 12))
style.configure("TButton", font=("Helvetica", 12))

# Create a Label for column selection
column_label = ttk.Label(root, text="Select a Target-Related Column:", font=("Helvetica", 14))
column_label.pack(pady=10)

# Create a Combobox for column selection
column_combobox = ttk.Combobox(root, values=columns, style="TCombobox")
column_combobox.pack(pady=10, ipadx=20, ipady=10)
column_combobox.bind("<<ComboboxSelected>>", on_select_column)

# Create a Label for term selection
term_label = ttk.Label(root, text="Select a Term:", font=("Helvetica", 14))
term_label.pack(pady=10)

# Create a Combobox for term selection
term_combobox = ttk.Combobox(root, style="TCombobox")
term_combobox.pack(pady=10, ipadx=20, ipady=10)
term_combobox.bind("<<ComboboxSelected>>", on_select_term)

# Create a Button to manually trigger search
search_button = tk.Button(root, text="Search", command=on_select_term, font=("Helvetica", 12))
search_button.pack(pady=10, ipadx=20, ipady=10)

# Run the main loop
root.mainloop()


#Provenance by Drug Name
#If you want to find provenance by drug name, CID, or molecule_chembl_id, run this script.

#Key Features of the Script
    #Dynamic Column and Term Selection: Users can select from three specific columns (drug_name, CID, molecule_chembl_id) related to drug information.
    #Data Fetch and Display: Upon selection, the script fetches data matching the selected term and displays it in the console. It also saves the results to a TSV file for offline analysis.
    #User Interface: This tool provides a graphical interface using Tkinter, simplifying database interactions for users with varying levels of technical expertise. It is particularly accessible to researchers and pharmacologists.

#Here's the Python script drug_query.py, tailored to allow users to interactively select and retrieve data focusing on columns related to drug information: drug_name, CID, and molecule_chembl_id. This script provides a user interface for easy navigation and data retrieval.


In [30]:
import mysql.connector as connector
import pandas as pd
import tkinter as tk
from tkinter import ttk

def fetch_unique_terms(column_name):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"SELECT DISTINCT {column_name} FROM tictac_genes_diseases_associations ORDER BY {column_name};"
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch all unique terms
            results = cursor.fetchall()
            
            # Convert to list
            unique_terms = [row[0] for row in results]
            return unique_terms
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)
        return []
    
    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            
def fetch_data_for_term(column_name, search_term):
    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",
            database="tictac_db"
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query
            query = f"""
            SELECT * FROM aact_studies WHERE nct_id IN
            (SELECT DISTINCT t.nct_id
                FROM tictac_genes_diseases_associations AS t
                JOIN aact_studies AS s ON t.nct_id = s.nct_id
                WHERE t.{column_name} LIKE '%{search_term}%');
            """
            
            # Execute the query
            cursor.execute(query)
            
            # Fetch the results
            results = cursor.fetchall()
            
            # Get column names
            column_names = [i[0] for i in cursor.description]
            
            # Convert to DataFrame
            df = pd.DataFrame(results, columns=column_names)
            
            # Display DataFrame
            print(df)
            
            # Save to TSV file
            df.to_csv(f"{search_term}_data.tsv", sep='\t', index=False)
            
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection is not None and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

def on_select_column(event=None):
    selected_column = column_combobox.get()
    if selected_column:
        unique_terms = fetch_unique_terms(selected_column)
        term_combobox['values'] = unique_terms
        term_combobox.set('')
    else:
        print("No column selected.")

def on_select_term(event=None):
    selected_column = column_combobox.get()
    selected_term = term_combobox.get()
    if selected_column and selected_term:
        fetch_data_for_term(selected_column, selected_term)
    else:
        print("No term selected.")

# Create the main window
root = tk.Tk()
root.title("Drug Query Selector")

# Column options
columns = ['drug_name', 'CID', 'molecule_chembl_id']

# Create a style for the combobox and button
style = ttk.Style()
style.configure("TCombobox", font=("Helvetica", 12))
style.configure("TButton", font=("Helvetica", 12))

# Create a Label for column selection
column_label = ttk.Label(root, text="Select a Drug-Related Column:", font=("Helvetica", 14))
column_label.pack(pady=10)

# Create a Combobox for column selection
column_combobox = ttk.Combobox(root, values=columns, style="TCombobox")
column_combobox.pack(pady=10, ipadx=20, ipady=10)
column_combobox.bind("<<ComboboxSelected>>", on_select_column)

# Create a Label for term selection
term_label = ttk.Label(root, text="Select a Term:", font=("Helvetica", 14))
term_label.pack(pady=10)

# Create a Combobox for term selection
term_combobox = ttk.Combobox(root, style="TCombobox")
term_combobox.pack(pady=10, ipadx=20, ipady=10)
term_combobox.bind("<<ComboboxSelected>>", on_select_term)

# Create a Button to manually trigger search
search_button = tk.Button(root, text="Search", command=on_select_term, font=("Helvetica", 12))
search_button.pack(pady=10, ipadx=20, ipady=10)

# Run the main loop
root.mainloop()


MySQL connection is closed
Empty DataFrame
Columns: [nct_id, study_type, source, phase, overall_status, start_date, completion_date, enrollment, official_title]
Index: []
MySQL connection is closed


#Query by Both Biological Target and Disease name
##Provenance by Both Biological Target and Disease name
    #You may want to find provenance based on both target_name and disease_name.
    #This script connects to the database and allows users to fetch and save evidence data based on a specified target name and disease name. 
    #It retrieves relevant records from the aact_studies table. The script prompts users to enter two things sequentially: 
    #target and disease names, displaying results, and saving them accordingly. 
    #If no input is provided, the connection is automatically closed after 20 seconds of inactivity. 
    #Additionally, it prints server information, verifies the connection, and displays the tables in the database.


In [ ]:
import mysql.connector as connector
from mysql.connector import Error
import threading
import pandas as pd
import os

# Global variable to keep track of the connection
connection = None
cursor = None

def fetch_study_data(target_name, disease_name):
    global connection, cursor
    try:
        # Execute the specific query
        query = """
        SELECT * FROM aact_studies WHERE nct_id IN
        (SELECT DISTINCT t.nct_id
            FROM tictac_genes_diseases_associations AS t
            JOIN aact_studies AS s ON t.nct_id = s.nct_id
            WHERE t.tcrdTargetName LIKE %s
            AND t.disease_name = %s);
        """
        cursor.execute(query, (f'%{target_name}%', disease_name))
        results = cursor.fetchall()

        # Get column names
        column_names = [desc[0] for desc in cursor.description]

        # Create a DataFrame
        df = pd.DataFrame(results, columns=column_names)
        print(df)

        # Save the DataFrame to a TSV file
        file_name = f'results_{target_name}_{disease_name}.tsv'.replace(' ', '_')
        df.to_csv(file_name, sep='\t', index=False)
        print(f"Results saved to {file_name}")

    except Error as e:
        print("Error while executing the query", e)

def close_connection():
    global connection, cursor
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed due to inactivity.")

def prompt_for_next_query():
    global connection, cursor
    while True:
        target_name = input("Enter the target name (or press Enter to exit): ")
        if not target_name:
            print("No input provided. Closing the connection in 20 seconds...")
            timer = threading.Timer(20, close_connection)
            timer.start()
            timer.join()
            break
        disease_name = input("Enter the disease name: ")
        fetch_study_data(target_name, disease_name)

if __name__ == "__main__":
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",         # Replace with your actual host if different
            user="root",              # Replace with your actual username
            password="xxxxxx",         # Replace with your actual password
            database="tictac_db"      # Replace with your actual database name
        )

        if connection.is_connected():
            # Fetch and print some server information
            db_info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_info)

            # Create a cursor object
            cursor = connection.cursor()

            # Execute a simple query to verify the connection
            cursor.execute("SELECT DATABASE();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)

            # Optional: Show tables in the connected database
            cursor.execute("SHOW TABLES;")
            tables = cursor.fetchall()
            print("Tables in the database:", tables)

            # Prompt for the next query
            prompt_for_next_query()

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")


#User Input: The script begins by prompting the user to enter the gene symbol. This allows the script to be used interactively without modifying the code for different genes.
#Database Query: The query now includes the DISTINCT keyword to ensure that only unique idgTDL values are returned for the specified gene. This prevents duplicate entries from appearing in the output.
#Execution Flow: After establishing a database connection, the script executes the query using the user-provided gene symbol. It then prints out each unique idgTDL value associated with the gene.
#Error Handling and Cleanup: Proper error handling ensures that any issues during the database connection or query execution are reported. The script also makes sure to close the database connection and cursor properly in the finally block.
#This approach enhances the script's usability by allowing dynamic input and ensuring concise, duplicate-free output.

In [ ]:
import mysql.connector as connector

def fetch_idgTDL_for_gene():
    # Prompt the user to enter a gene symbol
    gene_name = input("Please enter the gene symbol: ")

    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",  # Modify as per your MySQL setup
            database="tictac_db"  # Your specific database name
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query to fetch distinct idgTDL values for the entered gene symbol
            query = """
            SELECT DISTINCT gene_symbol, idgTDL
            FROM tictac_genes_diseases_associations
            WHERE gene_symbol = %s;
            """
            
            # Execute the query with user input
            cursor.execute(query, (gene_name,))
            
            # Fetch the results
            results = cursor.fetchall()

            if results:
                for row in results:
                    print(f"Gene: {row[0]}, idgTDL: {row[1]}")
            else:
                print(f"No idgTDL information found for gene: {gene_name}")
    
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    fetch_idgTDL_for_gene()


##IDGTDL Look up
#This script allows the user to select which targeted_related column they want to query, in addition to entering the search term letting users perform searches on any of the specified columns: tcrdTargetName, uniprot, gene_symbol, or target_chembl_id.

#User Flexibility: The script starts by displaying the available columns for querying and then prompts the user to select one of these columns and provide a corresponding search term.
#Dynamic Query Execution: Based on user input, the script constructs and executes a SQL query to fetch distinct idgTDL values for the specified search term in the chosen column.
#Generalization: By allowing users to specify the column, the script is versatile and can be used for a variety of searches within the database, making it highly adaptable to different analytical needs.
#This version of the script is user-friendly and adaptable, making it suitable for various types of database queries within the specified columns.

#User Input: The script begins by prompting the user to enter the gene symbol. This allows the script to be used interactively without modifying the code for different genes.
#Database Query: The query now includes the DISTINCT keyword to ensure that only unique idgTDL values are returned for the specified gene. This prevents duplicate entries from appearing in the output.
#Execution Flow: After establishing a database connection, the script executes the query using the user-provided gene symbol. It then prints out each unique idgTDL value associated with the gene.
#Error Handling and Cleanup: Proper error handling ensures that any issues during the database connection or query execution are reported. The script also makes sure to close the database connection and cursor properly in the finally block.
#This approach enhances the script's usability by allowing dynamic input and ensuring concise, duplicate-free output.

In [ ]:

import mysql.connector as connector

def fetch_unique_value_for_column():
    # Prompt the user to enter a column name and a search term
    print("Available columns: tcrdTargetName, uniprot, gene_symbol, target_chembl_id")
    column_name = input("Please enter the column name from the list above: ")
    search_term = input(f"Please enter the search term for {column_name}: ")

    connection = None
    try:
        # Establish the connection
        connection = connector.connect(
            host="localhost",
            user="root",
            password="xxxxxx",  # Modify as per your MySQL setup
            database="tictac_db"  # Your specific database name
        )
        
        if connection.is_connected():
            # Create a cursor object
            cursor = connection.cursor()
            
            # Define the query to fetch distinct idgTDL values for the entered column and search term
            query = f"""
            SELECT DISTINCT {column_name}, idgTDL
            FROM tictac_genes_diseases_associations
            WHERE {column_name} = %s;
            """
            
            # Execute the query with user input
            cursor.execute(query, (search_term,))
            
            # Fetch the results
            results = cursor.fetchall()

            if results:
                for row in results:
                    print(f"{column_name}: {row[0]}, idgTDL: {row[1]}")
            else:
                print(f"No idgTDL information found for {column_name}: {search_term}")
    
    except connector.Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    fetch_unique_value_for_column()
